In [1]:
from tqdm import tqdm
import pandas as pd
import polars as pl
from edgar import set_identity
set_identity("836473263@qq.com")
from edgar import *
import time
import pickle
import sys
import os
import traceback

[01:36:50] INFO     Identity of the Edgar REST client set to [836473263@qq.com]                         ]8;id=675209;file:///home/jason/anaconda3/lib/python3.10/site-packages/edgar/core.py\core.py]8;;\:]8;id=614951;file:///home/jason/anaconda3/lib/python3.10/site-packages/edgar/core.py#136\136]8;;\

In [2]:
START_YEAR = 2016
END_YEAR = 2016
FILE_NAME = "filing_attachments"

In [3]:
filing_10k_by_year = {}
for year in range(START_YEAR, END_YEAR+1):
    # filing_10k = get_filings(year=year).filter(form=["10-K"])
    time.sleep(1)
    filing_10k = get_filings(year=year).filter(form=["10-K"]).latest(50)
    filing_10k_by_year[year] = filing_10k #?

[01:36:55] INFO     HTTP Request: GET                                                               ]8;id=399623;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=337221;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/full-index/2016/QTR3/form.gz "HTTP/1.1 200                  
                    OK"                                                                                            

           INFO     HTTP Request: GET                                                               ]8;id=832025;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=480732;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/full-index/2016/QTR4/form.gz "HTTP/1.1 200                  
                    OK"                                                                                            

           INFO     HTTP Request: GET                                                               ]8;id=286026;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=587170;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/full-index/2016/QTR2/form.gz "HTTP/1.1 200                  
                    OK"                                                                                            

[01:36:56] INFO     HTTP Request: GET                                                               ]8;id=634619;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=465436;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/full-index/2016/QTR1/form.gz "HTTP/1.1 200                  
                    OK"                                                                                            

In [4]:
def contains_trade_secret_column(df):
    # List of column names to search for
    search_columns = [
        "trade secret",
        "trade secrets",
        "net trade secret",
        "net trade secrets",
        "trade secrets (not subject to amortization)",
        "trade secrecy",
        "net trade secrecy",
        "know",
        "intelect",
        "confiden",
        "propriet"
    ]

    # Filter rows where the first column is a string, then check if any value matches the search_columns list
    filtered_df = df[df.iloc[:, 0].apply(lambda x: isinstance(x, str))]  # ?
    result = None
    try:
        result = any(filtered_df.iloc[:, 0].str.lower().isin(search_columns))
    except:
        return None
    return result

def contains_trade_secret(content):
    # Check if 'trade secret' or 'trade secrecy' is in the content
    return 'trade secret' in content.lower() or 'trade secrecy' in content.lower()

In [5]:
def my_print(s, verbose=False):
    if verbose:
        print(s)

def download_attachments(filing_10k_by_year, file_name, year, start_from=0):
    try:
        file_path = f'{file_name}_{year}.pkl'
        filing_attachments = {}

        # Starting index for downloading attachments
        start_index = start_from
        # Loop through each filing in the provided list of 10-K filings, starting from start_index.
        for i, filing in enumerate(list(filing_10k_by_year[year])[start_index:], start=start_index):
            print(filing)
            try:
            # Check if the current filing has any attachments.
                if filing.attachments:
                    # Select the first attachment of the current filing.
                    first_attachment = filing.attachments[0]

                    # Implement exponential backoff for download attempts
                    success = False
                    attempt = 0
                    backoff_time = 1800  # 30 minutes in seconds
                    while attempt < 5 and not success:
                        try:
                            # Download the attachment and store it in the dictionary with its URL as the key.
                            filing_attachments[first_attachment.url] = first_attachment.download()
                            success = True
                        except Exception as e:
                            my_print(f"Failed to download {first_attachment.url}. Retrying in {backoff_time/60} minutes. Error: {e}")
                            time.sleep(backoff_time)
                            backoff_time *= 2  # Double the backoff time
                            attempt += 1

                    # Check if download attempts exceeded
                    if not success:
                        # Save the year and index of the failed filing and exit
                        with open('last_failed_index.txt', 'w') as f:
                            f.write(f"{year},{i}")
                        raise Exception(f"Maximum retries exceeded for {first_attachment.url}. Stopping and saving year {year} and index {i}.")

                    time.sleep(0.2)
            except Exception as e:
                print(str(e))
                print(f"year: {year}, index: {i}")
                raise(e)

        # Save the updated attachments
        with open(file_path, 'wb') as file:
            pickle.dump(filing_attachments, file)
    except Exception as e:
        traceback.format_exc()
        raise(e)

In [6]:
def load_attachments(file_name, year):
    # Open the specified file in read-binary mode.
    with open(f'{file_name}_{year}.pkl', 'rb') as file:
        # Load and deserialize the data from the file.
        loaded_data = pickle.load(file)
        # Return the deserialized data.
        return loaded_data

In [ ]:
"know",
        "intelect",
        "confiden",
        "propriet"

In [17]:
# Create an empty DataFrame with specified columns
results_df =  pd.DataFrame(columns=["company_name", "filing_date", "cik", "trade_secrets","indicator", "url", "accession_no"])
keywords = ['secre', 'know', 'intelect', 'confiden', 'proprie']

# Assuming filing_10k is a list of filings
for year in range(START_YEAR, END_YEAR+1):
    stored_attachments = load_attachments(FILE_NAME,year)
    for i, filing in enumerate(filing_10k_by_year[year]):
        filing_date = filing.filing_date
        # Note: write a script to scrape conformed_year_dict
        # conformed_year = conformed_year_dict.get(filing.accession_no)
        cik = filing.cik
        company_name = filing.company
        indicator = 0
        url = None
        accession_no = filing.accession_no

        if filing.attachments:
            list_df = None
            try:
                first_attachment_url = filing.attachments[0].url
                content = stored_attachments.get(first_attachment_url)
                url = first_attachment_url
                list_df = pd.read_html(content)

                if contains_trade_secret(content):
                    indicator = 1

            except Exception as e:
                print(f'Error happened while calling read_html() for {filing_date}, {company_name}: {str(e)}')
                list_df = []

            # Variable to store the DataFrame if found
            target_trade_secret_form = None

            # Iterate over each DataFrame in the list
            for form_df in list_df:
                # Check if the string 'trade secret' is present in the DataFrame (case-insensitive)
                if any(keyword in form_df.to_markdown().lower() for keyword in keywords):
                # if contains_trade_secret_column(form_df):
                    # If found, assign this DataFrame to variable 'a' and break the loop
                    target_trade_secret_form = form_df
                    break

            # Check if a DataFrame containing 'trade secret' was found
            if target_trade_secret_form is not None:
                # Convert the pandas DataFrame 'target_k10_form' to a Polars DataFrame for further processing
                target_trade_secret_form = pl.from_pandas(target_trade_secret_form)
                trade_secrets: float = None
                # Iterate over each row in the Polars DataFrame
                for form_row in target_trade_secret_form.rows():
                    # Check if the first element of the row is not null or empty
                    if form_row[0]:
                        # Check if the string 'secret' is present in the first element (case-insensitive)
                        if any(keyword in str(form_row[0]).lower() for keyword in keywords):
                            # Iterate over each element in the current row
                            for item in form_row:
                                try:
                                    # Attempt to convert the element to a float and check if it's greater than 0
                                    if float(item) > 0:
                                        first_element_lower = form_row[0].lower()
                                        if any(keyword in first_element_lower for keyword in keywords):
                                            trade_secrets = float(item)
                                            break
                                        # elif form_row[0].lower() == "trade secret" or form_row[0].lower() == "trade secrets" or form_row[0].lower() == "trade secrets (not subject to amortization)":
                                        else: 
                                            print(form_row[0])
                                            print("Unexpected case")
                                except ValueError:
                                    # If an error occurs (e.g., when the element cannot be converted to a float), continue to the next element
                                    continue
                                except TypeError:
                                    continue

                new_row = pd.DataFrame([{
                    "company_name": company_name,
                    "filing_date": filing_date,
                    "cik": cik,
                    "trade_secrets": trade_secrets,
                    #"net_trade_secrets": net_trade_secrets,
                    "indicator": indicator,
                    "url": url,
                    "accession_no": accession_no
                }])
                # Concatenate the new row to the results DataFrame
                results_df = pd.concat([results_df, new_row], ignore_index=True)
            else:
                new_row = pd.DataFrame([{
                    "company_name": company_name,
                    "filing_date": filing_date,
                    "cik": cik,
                    "trade_secrets": None,
                    #"net_trade_secrets": None,
                    "indicator": indicator,
                    "url": url,
                    "accession_no": accession_no
                }])
                # Concatenate the new row to the results DataFrame
                results_df = pd.concat([results_df, new_row], ignore_index=True)
                print(f"No DataFrame contains 'trade secret' in filing {i}.")

# Display the final results DataFrame
print(results_df)

[02:47:35] INFO     HTTP Request: GET                                                               ]8;id=513132;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=104820;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/779544/0000930413-16-009088-index.html                 
                    "HTTP/1.1 200 OK"                                                                              

           INFO     HTTP Request: GET                                                               ]8;id=290422;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=681948;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/827876/0001663577-16-000436-index.html                 
                    "HTTP/1.1 200 OK"                                                                              

[02:47:36] INFO     HTTP Request: GET                                                               ]8;id=486124;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=355055;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/1076682/0001654954-16-005166-index.html                
                    "HTTP/1.1 200 OK"                                                                              

           INFO     HTTP Request: GET                                                               ]8;id=516841;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=462482;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/1576873/0001078782-16-004032-index.html                
                    "HTTP/1.1 200 OK"                                                                              

           INFO     HTTP Request: GET                                                               ]8;id=116623;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=942939;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/1663641/0001393905-16-001132-index.html                
                    "HTTP/1.1 200 OK"                                                                              

...

No DataFrame contains 'trade secret' in filing 46.


           INFO     HTTP Request: GET                                                               ]8;id=300877;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=898042;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/96699/0001144204-16-141277-index.html                  
                    "HTTP/1.1 200 OK"                                                                              

           INFO     HTTP Request: GET                                                               ]8;id=737792;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=44859;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/794170/0000794170-16-000131-index.html                 
                    "HTTP/1.1 200 OK"                                                                              

[02:48:30] INFO     HTTP Request: GET                                                               ]8;id=441863;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=99491;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/1626696/0001052918-16-001474-index.html                
                    "HTTP/1.1 200 OK"                                                                              

                                company_name filing_date      cik  \
0                       ARK RESTAURANTS CORP  2016-12-30   779544   
1                           CLEANSPARK, INC.  2016-12-30   827876   
2                   MAJESCO ENTERTAINMENT CO  2016-12-30  1076682   
3                   Oroplata Resources, Inc.  2016-12-30  1576873   
4                      PC Mobile Media Corp.  2016-12-30  1663641   
5                            AMBER GROUP INC  2016-12-29  1621697   
6                           APP Incline Corp  2016-12-29  1669513   
7                               BALLY, CORP.  2016-12-29  1591565   
8                         Barrel Energy Inc.  2016-12-29  1631463   
9            Citizens Community Bancorp Inc.  2016-12-29  1367859   
10          Corning Natural Gas Holding Corp  2016-12-29  1582244   
11         First Savings Financial Group Inc  2016-12-29  1435508   
12                       GOLDEN GRAIN ENERGY  2016-12-29  1206942   
13                                

In [ ]:
# Create an empty DataFrame with specified columns
results_df =  pd.DataFrame(columns=["company_name", "filing_date", "cik", "trade_secrets", "net_trade_secrets","indicator", "url", "accession_no"])
keywords = ['secre', 'know', 'intelect', 'confiden', 'proprie']
# Assuming filing_10k is a list of filings
for year in range(START_YEAR, END_YEAR+1):
    stored_attachments = load_attachments(FILE_NAME,year)
    for i, filing in enumerate(filing_10k_by_year[year]):
        filing_date = filing.filing_date
        # Note: write a script to scrape conformed_year_dict
        # conformed_year = conformed_year_dict.get(filing.accession_no)
        cik = filing.cik
        company_name = filing.company
        indicator = 0
        url = None
        accession_no = filing.accession_no

        if filing.attachments:
            list_df = None
            try:
                first_attachment_url = filing.attachments[0].url
                content = stored_attachments.get(first_attachment_url)
                url = first_attachment_url
                list_df = pd.read_html(content)

                if contains_trade_secret(content):
                    indicator = 1

            except Exception as e:
                print(f'Error happened while calling read_html() for {filing_date}, {company_name}: {str(e)}')
                list_df = []

            # Variable to store the DataFrame if found
            target_trade_secret_form = None

            # Iterate over each DataFrame in the list
            for form_df in list_df:
                # Check if the string 'trade secret' is present in the DataFrame (case-insensitive)
                if 'trade secret' in form_df.to_markdown().lower():
                # if contains_trade_secret_column(form_df):
                    # If found, assign this DataFrame to variable 'a' and break the loop
                    target_trade_secret_form = form_df
                    break

            # Check if a DataFrame containing 'trade secret' was found
            if target_trade_secret_form is not None:
                # Convert the pandas DataFrame 'target_k10_form' to a Polars DataFrame for further processing
                target_trade_secret_form = pl.from_pandas(target_trade_secret_form)
                trade_secrets: float = None
                net_trade_secrets: float = None
                # Iterate over each row in the Polars DataFrame
                for form_row in target_trade_secret_form.rows():
                    # Check if the first element of the row is not null or empty
                    if form_row[0]:
                        # Check if the string 'secret' is present in the first element (case-insensitive)
                        if 'secret' in str(form_row[0]).lower():
                            # Iterate over each element in the current row
                            for item in form_row:
                                try:
                                    # Attempt to convert the element to a float and check if it's greater than 0
                                    if float(item) > 0:
                                        if form_row[0].lower() == "net trade secret" or form_row[0].lower() == "net trade secrets" or  form_row[0].lower() == "net trade secrecy":
                                            net_trade_secrets = float(item)
                                            break
                                        # elif form_row[0].lower() == "trade secret" or form_row[0].lower() == "trade secrets" or form_row[0].lower() == "trade secrets (not subject to amortization)":
                                        elif  "trade secret" in form_row[0].lower() or "trade secrecy"  in form_row[0].lower():
                                            try:
                                                trade_secrets = float(item)
                                            except Exception as e:
                                                print(f"Edge case: {filing_date} {cik} {company_name}") # debug only
                                                break
                                            break
                                        else:
                                            print(form_row[0])
                                            print("Unexpected case") # debug only
                                except ValueError:
                                    # If an error occurs (e.g., when the element cannot be converted to a float), continue to the next element
                                    continue
                                except TypeError:
                                    continue

                new_row = pd.DataFrame([{
                    "company_name": company_name,
                    "filing_date": filing_date,
                    "cik": cik,
                    "trade_secrets": trade_secrets,
                    "net_trade_secrets": net_trade_secrets,
                    "indicator": indicator,
                    "url": url,
                    "accession_no": accession_no
                }])
                # Concatenate the new row to the results DataFrame
                results_df = pd.concat([results_df, new_row], ignore_index=True)
            else:
                new_row = pd.DataFrame([{
                    "company_name": company_name,
                    "filing_date": filing_date,
                    "cik": cik,
                    "trade_secrets": None,
                    "net_trade_secrets": None,
                    "indicator": indicator,
                    "url": url,
                    "accession_no": accession_no
                }])
                # Concatenate the new row to the results DataFrame
                results_df = pd.concat([results_df, new_row], ignore_index=True)
                print(f"No DataFrame contains 'trade secret' in filing {i}.")

# Display the final results DataFrame
print(results_df)


In [ ]:
results_df.to_csv('result_df_1996_latest50.csv')

In [ ]:
import polars as pl

In [ ]:
df = pl.read_csv('filing_attachments:2016-2016.csv')

In [ ]:
df.describe()

In [ ]:
filtered_df = df.filter(pl.col('indicator') == 1)

In [ ]:
filtered_df